In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#infile = "../NTuples/test_truth.root"
#infile = "../NTuples/small_tpceta_sample.root"
#infile = "../NTuples/full_tpceta_sample.root"
#infile = "../NTuples/production_sample_take1.root"
infile = "../NTuples/production_sample_take2.root"

In [ ]:

inFile = uproot.open(infile)

#h_tot_pot = ROOT.TFile.Open(infile).Get("TotalPOT")
#pot = h_tot_pot.GetBinContent(1)

slc_tree = inFile["slc_tree"]
particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]
daughter_tree1 = inFile["daughter_tree1"]
daughter_tree2 = inFile["daughter_tree2"]
cosmic_tree1 = inFile["cosmic_tree1"]
cosmic_tree2 = inFile["cosmic_tree2"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")
daughter_df1 = daughter_tree1.arrays(daughter_tree1.keys(), library="pd")
daughter_df2 = daughter_tree2.arrays(daughter_tree2.keys(), library="pd")
cosmic_df1 = cosmic_tree1.arrays(cosmic_tree1.keys(), library="pd")
cosmic_df2 = cosmic_tree2.arrays(cosmic_tree2.keys(), library="pd")

slc_df[:2]

In [ ]:
particle_df1[:2]

In [ ]:
particle_df2[:2]

In [ ]:
daughter_df1[:2]

In [ ]:
daughter_df2[:2]

In [ ]:
cosmic_df1[:2]

In [ ]:
cosmic_df2[:2]

In [ ]:
cosmic_df1["KEEP"] = 0
condition = cosmic_df1.index.isin(cosmic_df1.query("pdg == 13 or pdg == -13").index)
cosmic_df1.loc[condition, "KEEP"] = 1
cosmic_df2["KEEP"] = cosmic_df1["KEEP"]

cosmic_df1 = cosmic_df1.query("KEEP == 1")
cosmic_df2 = cosmic_df2.query("KEEP == 1")

cosmic_df1["start_x"] = cosmic_df2["start_x"]
cosmic_df1["start_y"] = cosmic_df2["start_y"]
cosmic_df1["start_z"] = cosmic_df2["start_z"]
cosmic_df1["end_x"] = cosmic_df2["end_x"]
cosmic_df1["end_y"] = cosmic_df2["end_y"]
cosmic_df1["end_z"] = cosmic_df2["end_z"]

cosmic_df1[:2]

In [ ]:
def is_inTPC(row):
    x, y, z = row["vtx_x"], row["vtx_y"], row["vtx_z"]
    if math.isnan(x) or math.isnan(y) or math.isnan(z):
        return 0
    #if (x == 'NaN') or (y == 'NaN') or (z == 'NaN'):
    #    return 0
        
    if (-200 < x < 200) and (-200 < y < 200) and (0 < z < 500):
        return 1
    else:
        return 0

slc_df["inTPC"] = slc_df.apply(is_inTPC, axis=1)
slc_df[:2]

In [ ]:
topology_labels = {
    0:r"$\nu_{\mu} CC$",
    1:r"$\nu_{\mu} NC$",
    2:r"$\nu_{e} CC$",
    3:r"$\nu_{e} NC$",
    4:r"$\bar{\nu}_{\mu}$",
    5:r"$\bar{\nu}_{e}$",
    6:r"DIRT $\nu$",
    7:"Cosmic",
}

topology_selections = {
    0:"pdg == 14.0 and iscc == 1.0 and inTPC == 1",
    1:"pdg == 14.0 and isnc == 1.0 and inTPC == 1",
    2:"pdg == 12.0 and iscc == 1.0 and inTPC == 1",
    3:"pdg == 12.0 and isnc == 1.0 and inTPC == 1",
    4:"pdg == -14.0 and inTPC == 1",
    5:"pdg == -12.0 and inTPC == 1",
    6:"(pdg == 14.0 or pdg == 12.0 or pdg == -14.0 or pdg == -12.0 ) and inTPC == 0",
    7:"pdg == -1",
}

In [ ]:
# Let's look at the file composition

N = slc_df.shape[0]

bin_centers = [1]
bin_width = 1
N_prev = 0
#plt.yscale('log')
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    
    plt.bar(bin_centers, [N_top], width=bin_width, bottom=N_prev, alpha=0.6, label=topology_labels[num])
    N_prev += N_top

plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.xticks([])
plt.ylabel("Slice Count", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
# Let's add the topologies to the dataframes

# initialize to -1 for now
slc_df["TOP"] = -1

for num in range(len(topology_selections.keys())):
    #temp = slc_df.query(topology_selections[num])
    condition = slc_df.index.isin(slc_df.query(topology_selections[num]).index)
    slc_df.loc[condition, "TOP"] = num
    
slc_df[:2]


In [ ]:


#df_result = df_large.merge(df_small, on=['col1', 'col2'], how='left')

# Let's try to add the topology to the particle dfs
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP']]
particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

particle_df1[:2]

In [ ]:
daughter_df1 = daughter_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

daughter_df1[:2]

In [ ]:
particle_df2["TOP"] = particle_df1["TOP"]
daughter_df2["TOP"] = daughter_df1["TOP"]

particle_df2[:2]

In [ ]:

cosmic_primaries = particle_df2.query("TOP == 7")
cosmic_primaries[:2]

#plt.hist(particle_df2.query("TOP == 7")["start_x"].values, bins=100, label=topology_labels[7])
#plt.legend()
#plt.show()

In [ ]:
# Load the CRT histograms and make some geometry
WEST_X = 383
EAST_X = -383
SOUTH_Z = -179.05 
NORTH_Z = 779.85
FLAT_Y = -378.8
TOPLOW_Y = 632
TOPHIGH_Y = 767


In [ ]:
# implement the TPC Geometry 

NORTH_TPC_Z = 500
SOUTH_TPC_Z = 0
EAST_TPC_X = -200
WEST_TPC_X = 200
TOP_TPC_Y = 200
BOTT_TPC_Y = -200


def cross_top_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOP_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOP_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_bottom_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (BOTT_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (BOTT_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0


def cross_south_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0
        
def cross_north_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0


def cross_east_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_west_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0



def cross_tpc(start, dir):
    top = cross_top_tpc(start, dir)
    bott = cross_bottom_tpc(start, dir)
    north = cross_north_tpc(start, dir)
    south = cross_south_tpc(start, dir)
    east = cross_east_tpc(start, dir)
    west = cross_west_tpc(start, dir)
    if top or bott or north or south or east or west:
        return 1
    else:
        return 0
        

In [ ]:
def check_tpc_crossing(row):
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]
    if (-200 < end_x < 200) and (-200 < end_y < 200) and (0 < end_z < 500):
        return 1
        
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
    dir = [row["pc"]/norm, row["py"]/norm, row["pz"]/norm]
    cross = cross_tpc(start, dir)
    return cross
    
cosmic_df1["cross_tpc"] = cosmic_df1.apply(check_tpc_crossing, axis=1)
cosmic_df1[:2]

In [ ]:

bin_centers = [0, 1]
bin_width = 1
N_prev_sel = 0
N_prev_rej = 0

N_cross = cosmic_df1.query("cross_tpc == 1").shape[0]
N_not_cross = cosmic_df1.query("cross_tpc == 0").shape[0]
    
plt.bar(bin_centers, [N_cross, N_not_cross], width=bin_width, alpha=0.6, label="Cosmic Muons")

#plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.legend()
#plt.xticks([])
plt.xticks([0, 1], ['Crosses TPC', 'No TPC'])
plt.ylabel("Muon Count", fontsize=14)
#plt.ylim([0, N])
#plt.title("All Primary Particles")
plt.show()

In [ ]:
# Load the CRT histograms

crt_hist_file = ROOT.TFile.Open("/Users/alexanderantonakis/Desktop/crt_heatmaps.root", "READ")

h_tophigh = crt_hist_file.Get("h_tophigh_strip")
h_toplow = crt_hist_file.Get("h_toplow_strip")
h_north = crt_hist_file.Get("h_north_strip")
h_south = crt_hist_file.Get("h_south_strip")
h_east = crt_hist_file.Get("h_east_strip")
h_west = crt_hist_file.Get("h_west_strip")
h_flat = crt_hist_file.Get("h_flat_strip")

c = ROOT.TCanvas("c", "c", 700, 500)
h_flat.Draw("Colz")
c.Draw()

In [ ]:
# Implement the CRT geometry

def cross_crt_tophigh(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPHIGH_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPHIGH_Y - bzy)/mzy
    binx = h_tophigh.GetXaxis().FindBin(z_int)
    biny = h_tophigh.GetYaxis().FindBin(x_int)
    if h_tophigh.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_toplow(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPLOW_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPLOW_Y - bzy)/mzy
    binx = h_toplow.GetXaxis().FindBin(z_int)
    biny = h_toplow.GetYaxis().FindBin(x_int)
    if h_toplow.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_south(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_Z + byz
    binx = h_south.GetXaxis().FindBin(x_int)
    biny = h_south.GetYaxis().FindBin(y_int)
    if h_south.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_north(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_Z + byz
    binx = h_north.GetXaxis().FindBin(x_int)
    biny = h_north.GetYaxis().FindBin(y_int)
    if h_north.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_east(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_X - bxz)/mxz
    binx = h_east.GetXaxis().FindBin(z_int)
    biny = h_east.GetYaxis().FindBin(y_int)
    if h_east.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_west(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_X - bxz)/mxz
    binx = h_west.GetXaxis().FindBin(z_int)
    biny = h_west.GetYaxis().FindBin(y_int)
    if h_west.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_flat(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (FLAT_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (FLAT_Y - bzy)/mzy
    binx = h_flat.GetXaxis().FindBin(z_int)
    biny = h_flat.GetYaxis().FindBin(x_int)
    
    if h_flat.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt(start, dir):
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    num_c = c_tophigh + c_toplow + c_north + c_south + c_east + c_west + c_flat
    return num_c
    
print("made crt functions")

In [ ]:
def check_crt_crossing(row):
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]
    if (-200 < end_x < 200) and (-200 < end_y < 200) and (0 < end_z < 500):
        return 1
        
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
    dir = [row["pc"]/norm, row["py"]/norm, row["pz"]/norm]
    cross = cross_crt(start, dir)
    return cross
    
cosmic_df1["cross_crt"] = cosmic_df1.apply(check_crt_crossing, axis=1)
cosmic_df1[:2]

In [ ]:
N_cosmic_muon_inTPC = cosmic_df1.query("cross_tpc == 1").shape[0]

N_cosmic_muon_inCRT = cosmic_df1.query("cross_crt > 0 and cross_tpc == 1").shape[0]


plt.errorbar([1], [N_cosmic_muon_inTPC], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inTPC), fmt="o", c="black", label="Crosses TPC")
plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.legend()
plt.xticks([])
plt.ylabel("Cosmic Muon Count", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
def cross_crt_eff(row):
    c = int(row["cross_crt"])
    count = 0
    for num in range(c): 
        r = np.random.uniform()
        if r <= 0.95:
            count += 1
    return count
    
    
cosmic_df1["cross_crt_eff"] = cosmic_df1.apply(cross_crt_eff, axis=1)   
cosmic_df1[:2]

In [ ]:
N_cosmic_muon_inCRT_eff = cosmic_df1.query("cross_crt_eff > 1 and cross_tpc == 1").shape[0]
plt.errorbar([1], [N_cosmic_muon_inTPC], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inTPC), fmt="o", c="black", label="Crosses TPC")
plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT_eff), fmt="o", c="red", label="Crosses CRT: 95% (N > 1)")
plt.legend()
plt.xticks([])
plt.ylabel("Cosmic Muon Count", fontsize=14)
#plt.yscale('log')
plt.show()


N_cosmic_muon_inCRT_eff = cosmic_df1.query("cross_crt_eff > 1 and cross_tpc == 1").shape[0]
plt.errorbar([1], [1.0*N_cosmic_muon_inCRT/N_cosmic_muon_inTPC], xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label="Crosses CRT")
#plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff/N_cosmic_muon_inTPC], xerr=[0.5], yerr=[0.0], fmt="o", c="red", label="Crosses CRT: 95% (N > 1)")
plt.legend()
plt.xticks([])
plt.ylabel("Cosmic Muon Fraction", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:

BLAH

In [ ]:
def is_prim_cont(row):
    run, subrun, evt, slc = row["run"], row["subrun"], row["evt"], row["slc"]
    q = "run == "+str(run)+" and subrun == "+str(subrun) + " and evt == "+str(evt) + "and slc == "+str(slc)
    temp_df = particle_df2.query(q)
    x_s, y_s, z_s = temp_df["start_x"].values, temp_df["start_y"].values, temp_df["start_z"].values
    x_e, y_e, z_e = temp_df["end_x"].values, temp_df["end_y"].values, temp_df["end_z"].values
    for num in range(len(x_s)):
        if (x_s[num] > 200 or x_s[num] < -200):
            return 0
        if (x_e[num] > 200 or x_e[num] < -200):
            return 0
        if (y_s[num] > 200 or y_s[num] < -200):
            return 0
        if (y_e[num] > 200 or y_e[num] < -200):
            return 0
        if (z_s[num] > 500 or z_s[num] < 0):
            return 0
        if (z_e[num] > 500 or z_e[num] < 0):
            return 0
        return 1


  
slc_df["primCont"] = slc_df.apply(is_prim_cont, axis=1)    

slc_df[:2]

In [ ]:
# Let's look at the file composition


bin_centers = [0, 1]
bin_width = 1
N_prev_sel = 0
N_prev_rej = 0
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    N_sel = temp_df.query("primCont == 1").shape[0]
    N_rej = N_top - N_sel
    
    plt.bar(bin_centers, [N_sel, N_rej], width=bin_width, bottom=[N_prev_sel, N_prev_rej], 
            alpha=0.6, label=topology_labels[num])
    N_prev_sel += N_sel
    N_prev_rej += N_rej

#plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.legend()
#plt.xticks([])
plt.xticks([0, 1], ['Contained in TPC', 'Not Contained'])
plt.ylabel("Slice Count", fontsize=14)
plt.ylim([0, N])
plt.title("All Primary Particles")
plt.show()